# WosClient

In [1]:
# !pip install suds-jurko

from suds import client
from base64 import b64encode as _b64encode
from collections import OrderedDict as _OrderedDict

class WosClient():
    """Query the Web of Science.
       You must provide user and password only to user premium WWS service.
       with WosClient() as wos:
           results = wos.search(...)"""

    base_url = 'http://search.webofknowledge.com'
    auth_url = base_url + '/esti/wokmws/ws/WOKMWSAuthenticate?wsdl'
    search_url = base_url + '/esti/wokmws/ws/WokSearch?wsdl'
    searchlite_url = base_url + '/esti/wokmws/ws/WokSearchLite?wsdl'

    def __init__(self, user=None, password=None, SID=None, close_on_exit=True,
                 lite=False):
        """Create the SOAP clients. user and password for premium access."""

        self._SID = SID
        self._close_on_exit = close_on_exit
        search_wsdl = self.searchlite_url if lite else self.search_url
        self._auth = client.Client(self.auth_url)
        self._search = client.Client(search_wsdl)

        if user and password:
            auth = '%s:%s' % (user, password)
            auth = _b64encode(auth.encode('utf-8')).decode('utf-8')
            headers = {'Authorization': ('Basic %s' % auth).strip()}
            self._auth.set_options(headers=headers)

    def __enter__(self):
        """Automatically connect when used with 'with' statements."""
        self.connect()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        """Close connection after closing the 'with' statement."""
        if self._close_on_exit:
            self.close()

    def __del__(self):
        """Close connection when deleting the object."""
        if self._close_on_exit:
            self.close()

    def connect(self):
        """Authenticate to WOS and set the SID cookie."""
        if not self._SID:
            self._SID = self._auth.service.authenticate()
            print('Authenticated (SID: %s)' % self._SID)

        self._search.set_options(headers={'Cookie': 'SID="%s"' % self._SID})
        self._auth.options.headers.update({'Cookie': 'SID="%s"' % self._SID})
        return self._SID

    def close(self):
        """Close the session."""
        if self._SID:
            self._auth.service.closeSession()
            self._SID = None

    def search(self, query, count=5, offset=1):
    # def search(self, query, offset=1):
        """Perform a query. Check the WOS documentation for v3 syntax."""
        if not self._SID:
            raise RuntimeError('Session not open. Invoke .connect() before.')

        qparams = _OrderedDict([('databaseId', 'WOS'),
                                ('userQuery', query),
                                ('queryLanguage', 'en')])

        rparams = _OrderedDict([('firstRecord', offset),
                                ('count', count),
                                ('sortField', _OrderedDict([('name', 'RS'),
                                                            ('sort', 'D')]))])

        return self._search.service.search(qparams, rparams)

In [2]:

from xml.etree import ElementTree as _ET
from xml.dom import minidom as _minidom
import re as _re

def single(wosclient, wos_query, xml_query=None, count=10, offset=1):
    """Perform a single Web of Science query and then XML query the results."""
    result = wosclient.search(wos_query, count, offset)
    print (result)
    xml = _re.sub(' xmlns="[^"]+"', '', result.records, count=1).encode('utf-8')
    if xml_query:
        xml = _ET.fromstring(xml)
        return [el.text for el in xml.findall(xml_query)]
    else:
        return _minidom.parseString(xml).toprettyxml()

def query(wosclient, wos_query, xml_query=None, count=10, offset=1, limit=100):
    """Query Web of Science and XML query results with multiple requests."""
    results = [single(wosclient, wos_query, xml_query, min(limit, count-x+1), x) for x in range(offset, count+1, limit)]
    if xml_query:
        return [el for res in results for el in res]
    else:
        pattern = _re.compile(r'.*?<records>|</records>.*', _re.DOTALL)
        return ('<?xml version="1.0" ?>\n<records>' +
                '\n'.join(pattern.sub('', res) for res in results) +
                '</records>')

def doi_to_wos(wosclient, doi):
    """Convert DOI to WOS identifier."""
    results = query(wosclient, 'DO=%s' % doi, './REC/UID', count=1)
    return results[0].lstrip('WOS:') if results else None

In [56]:
soap = WosClient(lite=True)
soap.connect()

Authenticated (SID: D2NONlVlSayJQfAP4sr)


D2NONlVlSayJQfAP4sr

# Exchange the Given and Family Name

In [30]:
searchnames = []
i=0
for line in open('Dismis_Acad_List_CLEAN.txt','r'):
    QueryString = line.strip()
    QueryString = "AU="+line.split(',')[0].split()[-1]+' '+line.split(',')[0].split()[0]
    print('QueryString:', QueryString)
    searchnames.append(QueryString)


QueryString: AU=AYDIN AYDIN
QueryString: AU=GILGIL ERDAL
QueryString: AU=HALAC METIN
QueryString: AU=YUCE ABDULHAKIM
QueryString: AU=KIRIS ABDULKADIR
QueryString: AU=KUCUKBAYRAK ABDULKADIR
QueryString: AU=YILDIRIM ABDULKADIR
QueryString: AU=CEKIN ABDULKADIR
QueryString: AU=SENGUN ABDULKADIR
QueryString: AU=BEDIR ABDULKERIM
QueryString: AU=AKPINAR ABDULLAH
QueryString: AU=BAL ABDULLAH
QueryString: AU=SENGUL ABDULLAH
QueryString: AU=DEMIR ABDULLAH
QueryString: AU=DOGAN ABDULLAH
QueryString: AU=EKIZER ABDULLAH
QueryString: AU=ERDIL ABDULLAH
QueryString: AU=ERGIN ABDULLAH
QueryString: AU=SECKIN ABDULLAH
QueryString: AU=KIRAN ABDULLAH
QueryString: AU=USTUN ABDULLAH
QueryString: AU=CINGU ABDULLAH
QueryString: AU=AKSOY ABDULLAH
QueryString: AU=TUNCER ABDULLAH
QueryString: AU=MUTLU ABDULLAH
QueryString: AU=KILDAN ABDULLAH
QueryString: AU=POS ABDULLAH
QueryString: AU=SONMEZ ABDULLAH
QueryString: AU=URAL ABDULLAH
QueryString: AU=YORDEM ABDULLAH
QueryString: AU=TARIKULU ABDULLAH
QueryString: AU=C

QueryString: AU=LEBE FUAT
QueryString: AU=OZDINC FUAT
QueryString: AU=ATACAN FULYA
QueryString: AU=KENTLI FULYA
QueryString: AU=DOGRUEL FULYA
QueryString: AU=CATIR FULYA
QueryString: AU=OZDEMIR FUNDA
QueryString: AU=BIRDAL FURKAN
QueryString: AU=DINCER FURKAN
QueryString: AU=AYDOGAN FUSUN
QueryString: AU=SAHIN FIGEN
QueryString: AU=KOCYIGIT FIGEN
QueryString: AU=SELIMOGLU FIGEN
QueryString: AU=TASER FIGEN
QueryString: AU=ELMA FIKRET
QueryString: AU=KOCABAS FIKRET
QueryString: AU=KUYUCU FIKRET
QueryString: AU=MAZI FIKRET
QueryString: AU=UYAR FIKRET
QueryString: AU=DEMIR FIKRI
QueryString: AU=CEVIK FILIZ
QueryString: AU=OZDEMIR FILIZ
QueryString: AU=OZDEMIROGLU FILIZ
QueryString: AU=OZKAN FILIZ
QueryString: AU=IYIDIL FILIZ
QueryString: AU=CETIN FIRDEVS
QueryString: AU=SONMEZGOZ FITNET
QueryString: AU=OTURANC GALIP
QueryString: AU=CAG GALIP
QueryString: AU=YUCEL GERCEK
QueryString: AU=OZKUK GOKAN
QueryString: AU=ALPAK GOKAY
QueryString: AU=ERBAG GOKHAN
QueryString: AU=KARS GOKHAN
QueryStr

QueryString: AU=ACAR MUSTAFA
QueryString: AU=AKCAY MUSTAFA
QueryString: AU=SARIYILDIZ MUSTAFA
QueryString: AU=ALDEMIR MUSTAFA
QueryString: AU=ANAR MUSTAFA
QueryString: AU=ARIKAN MUSTAFA
QueryString: AU=ATASOY MUSTAFA
QueryString: AU=AKDEMIR MUSTAFA
QueryString: AU=AYDIN MUSTAFA
QueryString: AU=BAKIR MUSTAFA
QueryString: AU=BAYTAR MUSTAFA
QueryString: AU=BASER MUSTAFA
QueryString: AU=BENEKLI MUSTAFA
QueryString: AU=BIRINCIOGLU MUSTAFA
QueryString: AU=BICER MUSTAFA
QueryString: AU=CEMEK MUSTAFA
QueryString: AU=KARACADAG MUSTAFA
QueryString: AU=CENGIZ MUSTAFA
QueryString: AU=IZGI MUSTAFA
QueryString: AU=HOSCOSKUN MUSTAFA
QueryString: AU=DEMETGUL MUSTAFA
QueryString: AU=DEMIR MUSTAFA
QueryString: AU=DEMIRCI MUSTAFA
QueryString: AU=DEMIRCI MUSTAFA
QueryString: AU=DEMIREL MUSTAFA
QueryString: AU=DENIZ MUSTAFA
QueryString: AU=YILDIRIM MUSTAFA
QueryString: AU=DOGAN MUSTAFA
QueryString: AU=DURMAN MUSTAFA
QueryString: AU=DURMUS MUSTAFA
QueryString: AU=DONMEZ MUSTAFA
QueryString: AU=ERAYMAN MUSTA

QueryString: AU=BAYRAK OMER
QueryString: AU=BAYRAM OMER
QueryString: AU=BASAR OMER
QueryString: AU=CEVIT OMER
QueryString: AU=DEMIRTAS OMER
QueryString: AU=DEMIRTAS OMER
QueryString: AU=EROGLU OMER
QueryString: AU=EVIRGEN OMER
QueryString: AU=KOKOGLU OMER
QueryString: AU=SELAMET OMER
QueryString: AU=VURAL OMER
QueryString: AU=OZEKEN OMER
QueryString: AU=UNAL OMER
QueryString: AU=FARUKNOYAN OMER
QueryString: AU=GECICI OMER
QueryString: AU=KARAKAS OMER
QueryString: AU=KARATAS OMER
QueryString: AU=KURT OMER
QueryString: AU=MERMER OMER
QueryString: AU=PAMUK OMER
QueryString: AU=SAGLAM OMER
QueryString: AU=SOLAK OMER
QueryString: AU=SOYKASAP OMER
QueryString: AU=ULUKAPI OMER
QueryString: AU=UYSAL OMER
QueryString: AU=UCAR OMER
QueryString: AU=YAZGAN OMER
QueryString: AU=YILDIZ OMER
QueryString: AU=YILMAZ OMER
QueryString: AU=CAKIR OMER
QueryString: AU=CALKA OMER
QueryString: AU=CELIK OMER
QueryString: AU=OZBULUT OMER
QueryString: AU=OZKAN OMER
QueryString: AU=SAYLI OMER
QueryString: AU=SISE

In [35]:
len(searchnames)

3575

In [38]:
import pandas as pd

In [80]:
pd.DataFrame(searchnames,columns=['SearchNames']).to_csv('SearchNames.csv')

In [81]:
SearchNames = pd.read_csv('SearchNames.csv')

In [82]:
SearchNames.SearchNames[0].strip().split('=')[1]

'AYDIN AYDIN'

In [57]:
names = []
lens = []
i=0
for i in range(len(SearchNames)):
    line = SearchNames.SearchNames[i]
    QueryString = line.strip()
    print(QueryString)
    if i == 2400:
        soap = WosClient(lite=True)
        soap.connect()
    results = soap.search(QueryString)
    print(results.recordsFound)
    names.append(QueryString.split('=')[1])
    lens.append(results.recordsFound)
    time.sleep(0.5)
    i=i+1

AU=AYDIN AYDIN
4
AU=GILGIL ERDAL
5
AU=HALAC METIN
63
AU=YUCE ABDULHAKIM
0
AU=KIRIS ABDULKADIR
54
AU=KUCUKBAYRAK ABDULKADIR
30
AU=YILDIRIM ABDULKADIR
33
AU=CEKIN ABDULKADIR
0
AU=SENGUN ABDULKADIR
13
AU=BEDIR ABDULKERIM
38
AU=AKPINAR ABDULLAH
38
AU=BAL ABDULLAH
26
AU=SENGUL ABDULLAH
3
AU=DEMIR ABDULLAH
54
AU=DOGAN ABDULLAH
112
AU=EKIZER ABDULLAH
13
AU=ERDIL ABDULLAH
5
AU=ERGIN ABDULLAH
3
AU=SECKIN ABDULLAH
1
AU=KIRAN ABDULLAH
0
AU=USTUN ABDULLAH
5
AU=CINGU ABDULLAH
46
AU=AKSOY ABDULLAH
1
AU=TUNCER ABDULLAH
0
AU=MUTLU ABDULLAH
0
AU=KILDAN ABDULLAH
2
AU=POS ABDULLAH
0
AU=SONMEZ ABDULLAH
22
AU=URAL ABDULLAH
1
AU=YORDEM ABDULLAH
0
AU=TARIKULU ABDULLAH
0
AU=CIM ABDULLAH
14
AU=INALCIK ABDULLAH
5
AU=ASLAN ABDULSELAM
8
AU=GOK ABDULVAHAP
10
AU=OZTURK ABDURAHMAN
0
AU=ABAKAY ABDURRAHMAN
34
AU=DALGIC ABDURRAHMAN
0
AU=ELMALI ABDURRAHMAN
1
AU=GENC ABDURRAHMAN
21
AU=KADAYIFCI ABDURRAHMAN
53
AU=KUZHAN ABDURRAHMAN
4
AU=KIREMITCI ABDURRAHMAN
20
AU=BODUR ABDURRAHMAN
4
AU=SAHIN ABDURRAHMAN
64
AU=ELDEMIR ABD

AU=KORKMAZ AYSE
58
AU=SAGIROGLU AYSE
9
AU=SEZGIN AYSE
7
AU=KURT AYSE
12
AU=ULU AYSE
1
AU=YIGIT AYSE
19
AU=KIZILKAN AYSE
0
AU=SARAC AYSEGUL
14
AU=KALKAN AYSEGUL
0
AU=TURKYILMAZ AYSEGUL
30
AU=KURNAZ AYSEGUL
0
AU=AYSEUMUL AYSEUMUL
0
AU=ULUDAG AYSEUL
0
AU=ILBAY AZMI
1
AU=ILHAN AZIME
1
AU=ASAN AZIZ
0
AU=GUMUS AZIZ
17
AU=HARMAN AZIZ
6
AU=KILINC AZIZ
2
AU=MERHAN AZIZ
2
AU=POLAT AZIZ
31
AU=TUNC AZIZ
3
AU=TURKOGLU AZIZ
19
AU=ARTIS AISE
1
AU=GENC ASIR
10
AU=ORENC ASIR
0
AU=YILDIRIM ASKIN
0
AU=SEKER ASKIN
43
AU=OZDEMIR ASUR
0
AU=GUNEY BAHA
0
AU=BOZOGLAN BAHADIR
6
AU=KUMRAL BAHADIR
16
AU=TANTAY BAHADIR
0
AU=ERDINC BAHATTIN
13
AU=TUNC BAHATTIN
88
AU=ERMIS BAHRI
15
AU=ARSLAN BAHRIYE
5
AU=DURSUN BAHTIYAR
19
AU=SARIBOGA BAHTIYAR
13
AU=SARITAS BAKI
0
AU=TURKKAN BAKI
6
AU=ALTAN BANU
0
AU=KALE BANU
4
AU=UYSAL BANU
12
AU=IZGI BANU
0
AU=GUNAY BANIHAN
12
AU=COREKCI BARAM
0
AU=GENCER BARAN
37
AU=ASCI BARIS
3
AU=ERCAL BARIS
17
AU=GULHAN BARIS
12
AU=KIREMITCI BARIS
2
AU=YALINKILIC BARIS
5
AU=ALKAWASMEI BASSAM


AU=SAKALAR ERGUN
13
AU=BINGOLBALI ERHAN
11
AU=DELEN ERHAN
11
AU=GENC ERHAN
24
AU=GUNEYISI ERHAN
66
AU=KELESOGLU ERHAN
0
AU=METIN ERHAN
0
AU=SARI ERHAN
28
AU=TAHAN ERHAN
4
AU=VAROGLU ERHAN
29
AU=OZCAN ERHAN
14
AU=UNLU ERHAN
23
AU=KOMURCU ERKAM
30
AU=CURE ERKAN
51
AU=EFE ERKAN
13
AU=GOCEN ERKAN
4
AU=KAVAS ERKAN
0
AU=KACAN ERKAN
6
AU=KURT ERKAN
10
AU=SOGUT ERKAN
29
AU=YAMAN ERKAN
5
AU=YULA ERKAN
31
AU=CALISKAN ERKAN
10
AU=SIMSEK ERKAN
2
AU=BAGCIOGLU ERMAN
23
AU=GENC ERNUR
0
AU=ALBAYRAK EROL
5
AU=ERCAG EROL
25
AU=GUNDUZ EROL
1
AU=TOKSEN EROL
5
AU=CELIK EROL
0
AU=COSKUN EROL
25
AU=ONEN EROL
4
AU=CICEK ERSAN
14
AU=DAG ERSEL
27
AU=OBUZ ERSEL
16
AU=ERKAN ERSIN
0
AU=FADILLIOGLU ERSIN
32
AU=KARAPAZARLIOGLU ERSIN
0
AU=TERES ERSIN
1
AU=ERDEN ERSIN
9
AU=BESE ERTAN
0
AU=DURMUSOGLU ERTAN
16
AU=ERSOY ERTAN
0
AU=YANIK ERTAN
9
AU=OZTURK ERTAN
20
AU=ACARTURK ERTUGRUL
0
AU=ERCAN ERTUGRUL
49
AU=KARGI ERTUGRUL
15
AU=MAVILI ERTUGRUL
43
AU=UZUN ERTUGRUL
3
AU=BOZKURT ESAT
18
AU=CETIN ESAT
3
AU=SAVAS ESEN
24
AU

AU=BABACAN HASAN
22
AU=BAL HASAN
14
AU=ERTAS HASAN
25
AU=ULUSOY HASAN
40
AU=BAYRAKTAR HASAN
3
AU=BOZKURT HASAN
30
AU=BUKER HASAN
5
AU=BUYUKASLAN HASAN
14
AU=BILINC HASAN
13
AU=EKER HASAN
31
AU=ELLEK HASAN
0
AU=ERDOGAN HASAN
50
AU=ERDOGAN HASAN
50
AU=ERTAS HASAN
25
AU=ERCELIK HASAN
0
AU=EROL HASAN
2
AU=GOK HASAN
35
AU=GUNGOR HASAN
59
AU=ULUTIN HASAN
0
AU=DULGER HASAN
1
AU=HERKEN HASAN
65
AU=KART HASAN
3
AU=KOSGER HASAN
0
AU=SATILMIS HASAN
6
AU=YURTCU HASAN
1
AU=CALI HASAN
0
AU=CEVIK HASAN
3
AU=OZDEMIR HASAN
82
AU=YUCE HASAN
23
AU=KAFALI HASAN
52
AU=KALA HASAN
0
AU=KAMAK HASAN
11
AU=KAPLAN HASAN
26
AU=KARACAN HASAN
0
AU=KARAKURT HASAN
3
AU=KAYA HASAN
155
AU=KURT HASAN
15
AU=METINEREN HASAN
11
AU=MUTLU HASAN
83
AU=OZER HASAN
59
AU=SARAL HASAN
7
AU=GERGERLIOGLU HASAN
15
AU=TAK HASAN
1
AU=ATMACA HASAN
26
AU=TURKMEN HASAN
14
AU=UCMAK HASAN
18
AU=YILDI HASAN
0
AU=YILDIZHAN HASAN
6
AU=YUMAK HASAN
8
AU=CANKAYA HASAN
0
AU=CEVIKOGLU HASAN
0
AU=CECE HASAN
47
AU=CINAR HASAN
1
AU=OZTURK HASAN
74
AU=

AU=KAHRAMAN MEHMET
58
AU=KANDAZ MEHMET
55
AU=KANTER MEHMET
108
AU=KARAASLAN MEHMET
34
AU=KARABACAK MEHMET
60
AU=KARABIBER MEHMET
5
AU=KARACA MEHMET
159
AU=KARAKIZ MEHMET
2
AU=KARAKOC MEHMET
37
AU=KAYGANA MEHMET
0
AU=KAYRAK MEHMET
112
AU=KASLI MEHMET
3
AU=KARAAGAC MEHMET
2
AU=KIDIR MEHMET
0
AU=KILIC MEHMET
154
AU=KURTULMUS MEHMET
1
AU=KOROGLU MEHMET
63
AU=KIR MEHMET
22
AU=TASLI MEHMET
2
AU=OCAK MEHMET
6
AU=KUCUK MEHMET
56
AU=MELEK MEHMET
60
AU=MERCAN MEHMET
10
AU=METE MEHMET
2
AU=EROL MEHMET
43
AU=RIFAIOGLU MEHMET
9
AU=HAMIDI MEHMET
11
AU=NAM MEHMET
0
AU=NEBIOGLU MEHMET
12
AU=KUMRU MEHMET
7
AU=TURAN MEHMET
98
AU=KANBUR MEHMET
1
AU=OZYURTKAN MEHMET
18
AU=PAKDEMIRLI MEHMET
42
AU=PENCECIOGLU MEHMET
0
AU=HELVACI MEHMET
77
AU=KESICI MEHMET
1
AU=BURUNGUZ MEHMET
0
AU=DEMIRAY MEHMET
9
AU=ERSOY MEHMET
39
AU=AYDIN MEHMET
342
AU=ARBATLI MEHMET
0
AU=ARIK MEHMET
79
AU=SAKCALI MEHMET
6
AU=BAL MEHMET
16
AU=SUBASI MEHMET
17
AU=TAN MEHMET
34
AU=YASEMIN MEHMET
2
AU=TEKKOYUN MEHMET
5
AU=TEKINKUS MEHMET
1


AU=KUCUKONER MUSTAFA
0
AU=KIRKIT MUSTAFA
2
AU=LAMAN MUSTAFA
24
AU=MAMAK MUSTAFA
5
AU=TATLI MUSTAFA
44
AU=MENCUTEKIN MUSTAFA
0
AU=ARIKAN MUSTAFA
9
AU=METIN MUSTAFA
17
AU=ATANAK MUSTAFA
6
AU=MUJDECI MUSTAFA
0
AU=MUSTA MUSTAFA
3
AU=OBAY MUSTAFA
0
AU=OFLAZ MUSTAFA
0
AU=OGUZ MUSTAFA
27
AU=SINEMILLIOGLU MUSTAFA
0
AU=SAMANCIOGLU MUSTAFA
3
AU=SARICA MUSTAFA
0
AU=SACAR MUSTAFA
46
AU=SAGIR MUSTAFA
11
AU=SEKMEN MUSTAFA
1
AU=YAVUZ MUSTAFA
180
AU=SELCUK MUSTAFA
20
AU=KARA MUSTAFA
83
AU=KEMALOGLU MUSTAFA
2
AU=KARAKAS MUSTAFA
45
AU=SERINKEN MUSTAFA
62
AU=SUNBUL MUSTAFA
49
AU=TEKPINAR MUSTAFA
13
AU=TEKIN MUSTAFA
173
AU=ERDEM MUSTAFA
24
AU=BALTA MUSTAFA
2
AU=TOPARLI MUSTAFA
14
AU=TURKAL MUSTAFA
1
AU=TUTAK MUSTAFA
26
AU=GOKTAS MUSTAFA
25
AU=TURKMEN MUSTAFA
82
AU=TIMUR MUSTAFA
4
AU=ULASLI MUSTAFA
30
AU=UZKESER MUSTAFA
24
AU=UGURLU MUSTAFA
43
AU=VEKLI MUSTAFA
5
AU=YAPRAK MUSTAFA
36
AU=YAZICI MUSTAFA
35
AU=YESILYURT MUSTAFA
3
AU=YILDIRIM MUSTAFA
179
AU=YILDIRIM MUSTAFA
179
AU=YILDIRIM MUSTAFA
179
AU=YILDIZ

AU=SOGUT SADIK
15
AU=UCAR SADIK
0
AU=SAGLAM SADULLAH
16
AU=KARAMAN SAFA
56
AU=SEN SAFFET
1
AU=KOKSAL SAKIP
9
AU=AKCAY SALAHEDDIN
27
AU=AKYEL SALIH
1
AU=DEMIREL SALIH
43
AU=GORGUNOGLU SALIH
6
AU=KUK SALIH
29
AU=MUTLU SALIH
11
AU=OKUR SALIH
33
AU=SEYHAN SALIH
0
AU=TORUN SALIH
1
AU=YALCINBAS SALIH
19
AU=YILDIRIM SALIH
9
AU=YILMAZ SALIH
19
AU=CELEBIOGLU SALIH
5
AU=CELIK SALIH
33
AU=COSKUN SALIH
32
AU=OZ SALIH
0
AU=OZGOCMEN SALIH
87
AU=ILHAN SALIH
29
AU=INAL SALIH
47
AU=KARATAY SALIHA
44
AU=DONMEZ SALIM
42
AU=YUCE SALIM
45
AU=SAHIN SALIM
10
AU=KUVAT SAMET
71
AU=OZER SAMET
27
AU=KARAHAN SAMI
0
AU=TABAN SAMI
1
AU=BULUT SANCAR
10
AU=TEKIN SATI
3
AU=ALPAY SAVAS
0
AU=ACIKKAYA SAVAS
0
AU=OZSU SAVAS
57
AU=KARABULUT SAVAS
9
AU=KURT SAVAS
0
AU=SARIKAYA SAVAS
21
AU=ALGAN SAID
11
AU=ALMA SAIDE
0
AU=ARI SAIM
0
AU=BASKAN SAIM
3
AU=ALAN SAIT
31
AU=ERER SAIT
2
AU=SAN SAIT
43
AU=TUZEL SAIT
5
AU=CELIK SAIT
14
AU=AKKAYA SEBAHAT
0
AU=TURGUT SEBAHAT
51
AU=AGLADIOGLU SEBAHAT
30
AU=ALBAYRAK SEBAHATTIN
52
AU=GULT

AU=OZTURK VEYSI
4
AU=ERTURK VOLKAN
0
AU=SOLMAZ VOLKAN
36
AU=SENYUREK VOLKAN
12
AU=BOSNAK VUSLAT
20
AU=MOTOR VICDAN
13
AU=ZORBA YAHYA
17
AU=YASAR YAHYA
0
AU=CELIK YAHYA
53
AU=ISLAMOGLU YAHYA
34
AU=EGERCIOGLU YAKUP
9
AU=GONEN YAKUP
0
AU=KESKIN YAKUP
1
AU=SEKMEN YAKUP
6
AU=TURKEL YAKUP
19
AU=UMUCU YAKUP
7
AU=YILDIRIM YAKUP
68
AU=BAY YALCIN
0
AU=BAS YALCIN
24
AU=GOLCUK YALCIN
17
AU=YESIL YALCIN
8
AU=COPUR YALCIN
9
AU=ALPER YASEMIN
1
AU=BULUT YASEMIN
33
AU=KAYMAKCI YASEMIN
0
AU=GULUM YASEMIN
0
AU=ISIK YASEMIN
19
AU=BALCI YASEMIN
72
AU=KARACA YASEMIN
1
AU=TEKIN YASEMIN
1
AU=SOYORAL YASEMIN
36
AU=OZGUN YASEMIN
0
AU=ABUL YASIN
30
AU=ASLAN YASIN
0
AU=ATAC YASIN
2
AU=AVCI YASIN
1
AU=BEZ YASIN
53
AU=BULUT YASIN
6
AU=KARADAG YASIN
11
AU=KIRILMIS YASIN
0
AU=SARI YASIN
10
AU=SARIKAYA YASIN
9
AU=KARAKUS YASIN
16
AU=YAYLAR YASIN
0
AU=GULEN YASIR
4
AU=AKBAS YAVUZ
16
AU=GUNES YAVUZ
1
AU=KAPLAN YAVUZ
3
AU=SALIHOGLU YAVUZ
14
AU=SELIM YAVUZ
6
AU=TAMA YAVUZ
3
AU=SOYDAN YAVUZ
1
AU=UYAR YAVUZ
73
AU=UGURLU YAV

AU=BULU IRFAN
46
AU=DEGIRMENCI IRFAN
50
AU=ESER IRFAN
19
AU=KOCA IRFAN
50
AU=KUSCU IRFAN
0
AU=MUKUL IRFAN
0
AU=UCGUN IRFAN
6
AU=YILMAZ IRFAN
1
AU=SAKA IRFAN
7
AU=SIAP IRFAN
57
AU=BALDEMIR ISA
0
AU=DONER ISA
0
AU=SIYPAK ISA
0
AU=TASDELEN ISHAK
0
AU=PARMAKSIZ ISKENDER
16
AU=ACAR ISMAIL
7
AU=AKKAS ISMAIL
8
AU=AKSOY ISMAIL
20
AU=AKTAR ISMAIL
0
AU=ALBAYRAK ISMAIL
19
AU=ALTUNCU ISMAIL
0
AU=ARSEL ISMAIL
0
AU=ARSLAN ISMAIL
29
AU=ERDEN ISMAIL
58
AU=BOYACI ISMAIL
122
AU=BOYRAZ ISMAIL
22
AU=CAPAR ISMAIL
42
AU=DEMIR ISMAIL
83
AU=DOGAN ISMAIL
45
AU=GUNES ISMAIL
4
AU=EREN ISMAIL
3
AU=ERMIS ISMAIL
11
AU=ERTURK ISMAIL
35
AU=GUL ISMAIL
15
AU=GULCU ISMAIL
0
AU=UZUN ISMAIL
25
AU=UGURTAS ISMAIL
13
AU=KARA ISMAIL
55
AU=KATI ISMAIL
24
AU=KENAR ISMAIL
2
AU=KURNAZ ISMAIL
6
AU=KURT ISMAIL
49
AU=KOKSAL ISMAIL
16
AU=KULAHLI ISMAIL
6
AU=MANGALTEPE ISMAIL
0
AU=MARULCU ISMAIL
3
AU=METIN ISMAIL
1
AU=DOGAN ISMAIL
45
AU=UCUN ISMAIL
11
AU=ULUTAS ISMAIL
0
AU=ULUCAY ISMAIL
6
AU=VANLI ISMAIL
0
AU=YILDIZ ISMAIL
81
AU=YUKSE

In [68]:
df = pd.DataFrame([names,lens]).T
df.columns=['Author','NoPapers']

In [71]:
# 753 out of 3575 people have 0 papers in the web of science dataset
zeropapers = df.loc[df.NoPapers==0].copy()

In [72]:
# df.to_csv('Publishments.csv')

In [73]:
zeropapers.to_csv('ZeroPublishment.csv')

In [84]:
pd.concat([SearchNames,df],axis=1).to_csv('Publishments.csv')

In [85]:
pd.read_csv('Publishments.csv')

,Unnamed: 0,SearchNames,Author,NoPapers
0,0,AU=AYDIN AYDIN,AYDIN AYDIN,4
1,1,AU=GILGIL ERDAL,GILGIL ERDAL,5
2,2,AU=HALAC METIN,HALAC METIN,63
3,3,AU=YUCE ABDULHAKIM,YUCE ABDULHAKIM,0
4,4,AU=KIRIS ABDULKADIR,KIRIS ABDULKADIR,54
5,5,AU=KUCUKBAYRAK ABDULKADIR,KUCUKBAYRAK ABDULKADIR,30
6,6,AU=YILDIRIM ABDULKADIR,YILDIRIM ABDULKADIR,33
7,7,AU=CEKIN ABDULKADIR,CEKIN ABDULKADIR,0
8,8,AU=SENGUN ABDULKADIR,SENGUN ABDULKADIR,13
9,9,AU=BEDIR ABDULKERIM,BEDIR ABDULKERIM,38
